In [1]:
!pip install yt-dlp


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------  3.1/3.2 MB 16.9 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 14.4 MB/s eta 0:00:00


In [2]:

import yt_dlp as ytdlp

# YouTube video URL
url = "https://www.youtube.com/watch?v=o3xE32ovIMo"

def download_youtube_video(url, output_filename="time_lapse.mp4"):
    ydl_opts = {
        'format': 'mp4',  # Request MP4 format
        'outtmpl': output_filename,  # Specify output file
    }
    with ytdlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    print(f"Video downloaded as '{output_filename}'")
    return output_filename

# Download the video
download_youtube_video(url)


[youtube] Extracting URL: https://www.youtube.com/watch?v=o3xE32ovIMo
[youtube] o3xE32ovIMo: Downloading webpage
[youtube] o3xE32ovIMo: Downloading ios player API JSON
[youtube] o3xE32ovIMo: Downloading mweb player API JSON
[youtube] o3xE32ovIMo: Downloading player f8f53e1a
[youtube] o3xE32ovIMo: Downloading m3u8 information
[info] o3xE32ovIMo: Downloading 1 format(s): 18
[download] Destination: time_lapse.mp4
[download] 100% of    2.21MiB in 00:00:01 at 1.47MiB/s   
Video downloaded as 'time_lapse.mp4'


'time_lapse.mp4'

In [22]:
import cv2
import numpy as np

# Function to categorize brightness using fixed thresholds (HSV-based)
def categorize_brightness_fixed(frame):
    # Convert the frame to HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Extract the Value (V) channel which represents brightness
    v_channel = hsv_frame[:, :, 2]

    # Calculate the average brightness from the V channel
    brightness = np.mean(v_channel)  # Mean of pixel values in the V channel

    # Set fixed thresholds based on general assumptions
    if brightness > 90:  # High brightness: Day (higher than 90)
        return "Day", brightness
    elif brightness > 40:  # Moderate brightness: Morning (between 40 and 90)
        return "Morning", brightness
    elif brightness > 22:  # Moderate brightness: Evening (between 22 and 40)
        return "Evening", brightness
    else:  # Low brightness: Night (below 22)
        return "Night", brightness

# Input video file path
video_path = "time_lapse.mp4"
cap = cv2.VideoCapture(video_path)

# Initialize variables
day_count = 0
morning_count = 0
evening_count = 0
night_count = 0
total_frames = 0

# Set output file path and codec for AVI
output_path = "output_timelapse.avi"  # AVI format
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # XVID codec for AVI format

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Initialize the VideoWriter for saving the video
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Process the video frames and count occurrences of each category
while True:
    ret, frame = cap.read()
    if not ret:
        break  # If no frame is returned, break the loop  

    # Categorize the current frame using fixed thresholds
    category, brightness = categorize_brightness_fixed(frame)

    # Increment the appropriate counter based on the category
    if category == "Day":
        day_count += 1
    elif category == "Morning":
        morning_count += 1
    elif category == "Evening":
        evening_count += 1
    else:
        night_count += 1

    total_frames += 1  # Increment total frames count

    # Calculate percentages dynamically
    if total_frames > 0:
        day_percentage = (day_count / total_frames) * 100
        morning_percentage = (morning_count / total_frames) * 100
        evening_percentage = (evening_count / total_frames) * 100
        night_percentage = (night_count / total_frames) * 100
    else:
        day_percentage = morning_percentage = evening_percentage = night_percentage = 0

    # Prepare the text to display on the frame
    if category == "Day":
        display_text = f"Day: {day_percentage:.2f}%"
    elif category == "Morning":
        display_text = f"Morning: {morning_percentage:.2f}%"
    elif category == "Evening":
        display_text = f"Evening: {evening_percentage:.2f}%"
    else:
        display_text = f"Night: {night_percentage:.2f}%"

    # Add text to the frame (showing category and percentage)
    cv2.putText(frame, display_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write the updated frame to the output video
    out.write(frame)

# Release the video capture and writer objects
cap.release()
out.release()

# Close the display window
cv2.destroyAllWindows()

# Print final percentages for categories
if total_frames > 0:
    print(f"Morning: {morning_percentage:.2f}%")
    print(f"Day: {day_percentage:.2f}%")
    print(f"Evening: {evening_percentage:.2f}%")
    print(f"Night: {night_percentage:.2f}%")
else:
    print("No frames processed.")


Morning: 24.86%
Day: 51.22%
Evening: 5.21%
Night: 18.71%
